# Execute embedding and search on remote server

In [1]:
!pip install paramiko

## Transfer data file

In [11]:
import getpass
import subprocess
import os
from dotenv import load_dotenv
import ipywidgets as widgets
from IPython.display import display
import json

# Your code to be executed when the button is clicked
def send_data_to_server():
# Load environment variables from .env file
    load_dotenv()
    
    # Define a variable based on a value from .env
    server = os.getenv("server")
    username=os.getenv("username")
    remote_host=username+"@"+server
    filename = file_selector.value
    filename2 = file_selector2.value
    filename3="searchconfig.json"
    print(remote_host)
    conf={"filename_sdg":filename2,
          "filename_texts":filename,
          'Prompt':text1.value,
          'extracted_text':text2.value
         }
    with open('searchconfig.json', "w") as f:
        json.dump(conf, f, indent=4)
          
    remote_path = "/home/codechallenge/InnoClass/backend/classEngine/data/"
    #password = getpass.getpass(f"Enter password for {remote_host}: ")
    #command = ["scp", "-o", f"StrictHostKeyChecking=no", "-o", f"PreferredAuthentications=password", filename, f"{remote_host}:{remote_path}"]
    command = ["scp", filename, f"{remote_host}:{remote_path}"]
    command2 = ["scp", filename2, f"{remote_host}:{remote_path}"]
    command3 = ["scp", filename3, f"{remote_host}:{remote_path}"]
    try:
        print("command 1 : ")
        process = subprocess.Popen(command, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        stdout, stderr = process.communicate() # Append newline to password
        return_code = process.returncode
        if return_code == 0:
            print("SCP command executed successfully.")
            print("Output:", stdout)
        else:
            print(f"SCP command failed with error (code {return_code}):")
            print("Error!:", stderr)
        print("command 2 : ")
        process = subprocess.Popen(command2, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        stdout, stderr = process.communicate() # Append newline to password
        return_code = process.returncode
        if return_code == 0:
            print("SCP command executed successfully.")
            print("Output:", stdout)
        else:
            print(f"SCP command failed with error (code {return_code}):")
            print("Error!:", stderr)
        print("command 3 : ")
        process = subprocess.Popen(command3, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        stdout, stderr = process.communicate() # Append newline to password
        return_code = process.returncode
    
        if return_code == 0:
            print("SCP command executed successfully.")
            print("Output:", stdout)
        else:
            print(f"SCP command failed with error (code {return_code}):")
            print("Error!:", stderr)
    
    except FileNotFoundError:
        print("Error: scp command not found.")
    
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    print("Sending data to server...")


# Create the button
send_button = widgets.Button(description="Send to Server")

# Get list of files ending with ".sen.gz" from current directory
files = [f for f in os.listdir('.') if f.endswith('sen.gz')]
files2 = [f for f in os.listdir('.') if f.endswith('.dat')]

# Widget for listing files
file_selector = widgets.Select(
    options=files,
    description='File patents:',
    rows=10,
    layout=widgets.Layout(width='200px')
)

file_selector2 = widgets.Select(
    options=files2,
    description='File sdg targets:',
    rows=10,
    layout=widgets.Layout(width='200px')
)


text1=widgets.Text(value="Target Text",description="Prompt:")
text2=widgets.Text(value="sentence",description="Patent extracts:")

# Define the button's click event handler
def on_button_click(b):
    
    send_data_to_server()

# Attach the event handler to the button's click event
send_button.on_click(on_button_click)

# Display the widgets
display(send_button)
display(file_selector)
display(file_selector2)
display(text1)
display(text2)


Button(description='Send to Server', style=ButtonStyle())

Select(description='File patents:', layout=Layout(width='200px'), options=('test.sen.gz',), rows=10, value='te…

Select(description='File sdg targets:', layout=Layout(width='200px'), options=('sgd_goals.dat', 'sgd_targets.d…

Text(value='Target Text', description='Prompt:')

Text(value='sentence', description='Patent extracts:')

Configuration file for embedding and search

In [3]:
import paramiko
import os
from dotenv import load_dotenv
load_dotenv()


def execute_remote_command(hostname, username, private_key_path, command):
    """Executes a command on a remote machine using SSH keys."""
    try:
        private_key = paramiko.Ed25519Key.from_private_key_file(private_key_path)
        client = paramiko.SSHClient()
        client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        client.connect(hostname, username=username, pkey=private_key)

        stdin, stdout, stderr = client.exec_command(command)
        output = stdout.read().decode()
        error = stderr.read().decode()

        client.close()

        if error:
            print(f"Error: {error}")
        return output

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

execute_button = widgets.Button(description="Execute Search on Server")
# Define the button's click event handler
def on_execute_button_click(b):
    hostname = os.getenv("server")
    username = os.getenv("username")
    private_key_path = os.getenv("key")
    command = "cd /home/codechallenge/InnoClass/backend/; docker-compose up embedsearch"  # Replace with your desired command
    result = execute_remote_command(hostname,username, private_key_path,command)
    
    if result:
        print("Command output:")
        print(result)

# Attach the event handler to the button's click event
execute_button.on_click(on_execute_button_click)
display(execute_button)


Button(description='Execute Search on Server', style=ButtonStyle())